In [101]:
import pandas as pd

uri = "./machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


Separando os dados de treino e teste para realizar a comparação com outros modelos

In [102]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25,
                                                        stratify=y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


Antes de rodar o classificador, precisamos decidir um taxa de acerto aceitável.

Para se ter como base será utilizado um DUMMY CLASSIFIER, que é utilizado para se ter um fator de comparação com outros classificadores complexos. No DUMMY CLASSIFIER é realizado uma classificação através de regras simples. É de se esperar que qualquer outro classificador complexo terá de ser melhor que o DUMMY CLASSIFIER.

Estratégias utilizadas pelo DUMMY CLASSIFIER:

Mais frequente: o classificador sempre prevê o rótulo a classe mais frequente nos dados de treinamento
Estratificado: Gerar previsões respeitando a distribuição das classes dos dados de treino.
Uniforme: Gera previsões uniformemente de forma aleatória.
Constante: O classificador sempre prevê um rótulo constante e é usado principalmente ao classificar rótulos de classe não majoritária
A Taxa de acerto simples foi 58%

In [103]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A Acurácia do dummy stratified foi %.2f%%" % acuracia)

A Acurácia do dummy stratified foi 58.00%


Rodando a DecisionTreeClassifier

O problema é que cada vez que rodamos o algoritmo com um seed diferente, resulta em acuracidades diferentes. Um problema chamado de holdout, sendo arriscado para aplicações em modelos baseados na vida real.

In [104]:
from sklearn.tree import DecisionTreeClassifier 

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acuracia foi %.2f%%" % acuracia)

A acuracia foi 71.92%


Exemplo alterando o SEED

In [105]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25,
                                                       stratify=y)
#print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acuracia foi %.2f%%" % acuracia)

A acuracia foi 76.84%


Aplicando o cross_validate e testando as acuracidades com diferentes CV (Parâmetro que determina a quantidade de cortes para treino e teste), utilizando intervalos de confianças para deixar o modelos mais confiável.

In [106]:
from sklearn.model_selection import cross_validate
SEED = 12
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation, 3 = [74.99, 76.57]


O parâmetro utilizado será cv = 5, conforme o indicado pela documentação do sklearn.

[Doc.CrossValidation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

In [107]:
from sklearn.model_selection import cross_validate
SEED = 12
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation, 5 = [75.21, 76.35]


# Aleatoriedade no cross validate

KFold é um técnica de validação de modelo, onde é gerado de modo aleatório os valores de treino e teste, conforme é possível observar no exemplo abaixo

In [108]:
'''
# Não rodar para não alterar o desenvolver do código
import numpy as np
from sklearn.model_selection import KFold

x = [1,2,3,4,5,6,7,8,9,10,11,12]
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(x):
    print (train_index, test_index)
'''    

'\n# Não rodar para não alterar o desenvolver do código\nimport numpy as np\nfrom sklearn.model_selection import KFold\n\nx = [1,2,3,4,5,6,7,8,9,10,11,12]\nkf = KFold(n_splits=5)\n\nfor train_index, test_index in kf.split(x):\n    print (train_index, test_index)\n'

In [109]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media *100))
    print("Accuracy intervalo = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Técnica do KFold sem shuffle

In [110]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo = [74.37, 77.19]


Técnica do KFold com shuffle
Gera uma forma aleatória dos dados dentro do KFold (no intervalo dos dados)

In [111]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy intervalo = [73.26, 78.26]


# Simular situação horrível de azar

Pode ser "azar" como pode ser uma proporção de exemplos desbalanceado entre as classes.

In [112]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [113]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo = [34.29, 81.39]


In [114]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo = [72.30, 79.26]


In [115]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo = [73.55, 78.01]


In [116]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


# GroupKFold

Para criar modelos que são possuem variáveis separadas por grupos, é necessário utilizar a classe **GroupKFold**, que realiza a separação por agrupamento, buscando uma melhora na acurácia

In [117]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo = [73.83, 77.73]
